# Function for Cross Validation

In [1]:
def cv(model2):
    smote=SMOTE(sampling_strategy='minority',random_state=42)
    accuracy=[]
    precision=[]
    recall=[]
    f1_test=[]
    f1_train=[]
    l=1176

    for i in range(100):
        train=resample(new_df.values,n_samples=l)
        #print(train[0])
        tv=train.tolist()
        test=resample(np.array([i for i in new_df.values if i.tolist() not in tv]),
                      n_samples=(new_df.shape[0]-l))
        #print(test[0])
        X_sm, y_sm=smote.fit_sample(train[:,:-1],train[:,-1])
        model2.fit(X_sm,y_sm)
    
        #Appending Scores
    
        accuracy.append(metrics.accuracy_score(test[:,-1],model2.predict(test[:,:-1])))
        precision.append((metrics.precision_score(test[:,-1],model2.predict(test[:,:-1]))))
        recall.append((metrics.recall_score(test[:,-1],model2.predict(test[:,:-1]))))
        f1_test.append((metrics.f1_score(test[:,-1],model2.predict(test[:,:-1]))))
        f1_train.append((metrics.f1_score(y_sm,model2.predict(X_sm))))
    return(np.mean(accuracy).round(2),np.mean(precision).round(2),np.mean(recall).round(2),np.mean(f1_test).round(2))

# Function for manual Train-Test Split

In [3]:
def One_User_Out(data,model):
    '''  
    Input: dataframe , Model object
    
    Returns: Train_score,Test_score
    '''
    sc=StandardScaler()
    user=data.User.unique()
    test_score=[]
    train_score=[]
    for i in user:
        X_train=data[data.User!=i].drop(columns=['User','Class'])
        y_train=data[data.User!=i].Class
        X_test=data[data.User==i].drop(columns=['User','Class'])
        y_test=data[data.User==i].Class
        
        X_train_scaled=sc.fit_transform(X_train)
        X_test_scaled=sc.transform(X_test)
        
        model.fit(X_train_scaled,y_train)
        train_score.append(f1_score(y_train,model.predict(X_train_scaled),average='weighted'))
        
        test_score.append(f1_score(y_test,model.predict(X_test_scaled),average='weighted'))
    return(train_score,test_score)

In [4]:
def three_user_out_comb(model,data):
    from itertools import combinations
    from sklearn.metrics import f1_score
    import random  
    from sklearn.preprocessing import StandardScaler
    sc=StandardScaler()
    test_score=[]
    train_score=[]
    comb=combinations([0,1,2,5,6,8,9,10,11,12,13,14],3)
    lst=[]
    for i in comb:
        lst.append(list(i))
    np.random.shuffle(lst)
    l1=lst[:35]
    for i in l1:
        X_train=data[(data.User!=i[0])&(data.User!=i[1])&(data.User!=i[2])].drop(columns=['User','Class'])
        y_train=data[(data.User!=i[0])&(data.User!=i[1])&(data.User!=i[2])].Class
        X_test=data[(data.User==i[0])|(data.User==i[1])|(data.User==i[2])].drop(columns=['User','Class'])
        y_test=data[(data.User==i[0])|(data.User==i[1])|(data.User==i[2])].Class
        
        X_train_scaled=sc.fit_transform(X_train)
        X_test_scaled=sc.transform(X_test)
        model.fit(X_train_scaled,y_train)
        train_score.append(f1_score(y_train,model.predict(X_train_scaled),average='weighted'))
        test_score.append(f1_score(y_test,model.predict(X_test_scaled),average='weighted'))
    return(train_score,test_score)